In [1]:
import requests     
import json         
import pandas as pd 
from bs4 import BeautifulSoup
import json
import re
import csv
from io import StringIO
from datetime import datetime
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
os.chdir('C:\\Users\\fizpo')

In [3]:
Transactions = pd.read_csv('Transactions.csv')

In [4]:
Transactions.head(n = 2)

,Data,Czas,Produkt,ISIN,Giełda referenc,Miejsce wykonania,Liczba,Kurs,Unnamed: 8,Wartość lokalna,Unnamed: 10,Wartość,Unnamed: 12,Kurs wymian,Opłata transakcyjna,Unnamed: 15,Razem,Unnamed: 17,Identyfikator zlecenia
0,02-02-2022,17:31,PAYPAL HOLDINGS INC.,US70450Y1038,NDQ,CDED,1,131.75,USD,-131.75,USD,-116.63,EUR,1.1296,-0.5,EUR,-117.13,EUR,b9d097fb-1969-49dc-ab52-6e9ed379444e
1,04-01-2022,20:37,APPLE INC. - COMMON ST,US0378331005,NDQ,CDED,1,179.80,USD,-179.80,USD,-159.24,EUR,1.1291,-0.5,EUR,-159.74,EUR,5a614ead-62b9-47f8-b045-d15a7c14c99d


In [5]:
exchCode = [None]*len(Transactions)
Transactions['exchCode'] = exchCode
Transactions = Transactions.rename({'Data' : 'date', 'Czas':'time', 'Produkt': 'product','ISIN':'isin',
                                    'Giełda referenc':'stock_exchange', 'Miejsce wykonania':'execution_place',
                                    'Liczba':'number', 'Kurs':'price','Unnamed: 8':'trans_curr', 'Wartość lokalna':'local_value',
                                    'Unnamed: 10':'curr_local_value', 'Wartość' : 'price_eu', 
                                    'Unnamed: 12':'account_curr','Kurs wymian':'exchange_rate', 'Opłata transakcyjna':'commission',
                                    'Unnamed: 15':'commission_curr','Razem': 'total_eu', 'Unnamed: 17':'total_curr',
                                    'Identyfikator zlecenia': 'order_id'}, axis= 1)
Transactions = Transactions.drop('order_id', axis = 1)
Transactions['date'] = Transactions['date'].astype(str) + ' ' + Transactions['time'].astype(str)
Transactions['date'] = pd.to_datetime(Transactions['date'],dayfirst=True)
Transactions = Transactions.sort_values('date').reset_index(drop=True) 

In [6]:
Transactions.head()

,date,time,product,isin,stock_exchange,execution_place,number,price,trans_curr,local_value,curr_local_value,price_eu,account_curr,exchange_rate,commission,commission_curr,total_eu,total_curr,exchCode
0,2020-03-18 09:02:00,09:02,VANGUARD S&P500,IE00B3XXRP09,LSE,XLON,1,38.000,GBP,-38.00,GBP,-41.67,EUR,0.9119,-2.01,EUR,-43.68,EUR,None
1,2020-03-26 16:33:00,16:33,BANK OF AMERICA CORPOR,US0605051046,NSY,CDED,1,22.445,USD,-22.45,USD,-20.39,EUR,1.1010,-0.50,EUR,-20.89,EUR,None
2,2020-03-26 17:38:00,17:38,DELTA AIR LINES INC.,US2473617023,NSY,XNAS,1,35.000,USD,-35.00,USD,-31.75,EUR,1.1025,-0.50,EUR,-32.25,EUR,None
3,2020-03-30 15:30:00,15:30,DELTA AIR LINES INC.,US2473617023,NSY,XNYS,1,28.110,USD,-28.11,USD,-25.49,EUR,1.1028,-0.50,EUR,-25.99,EUR,None
4,2020-03-31 14:01:00,14:01,ISHRC S&P 500,IE00B5BMR087,LSE,XLON,1,260.000,USD,-260.00,USD,-237.33,EUR,1.0955,-2.07,EUR,-239.40,EUR,None


In [7]:
Transactions['isin'].unique()

array(['IE00B3XXRP09', 'US0605051046', 'US2473617023', 'IE00B5BMR087',
       'US0088751062', 'US00848K1016', 'US8762142060', 'US0400476075',
       'US80105N1054', 'US3755581036', 'US9216591084', 'US5949181045',
       'US44148G1058', 'US30303M1027', 'US89151E1091', 'US30231G1022',
       'US4581401001', 'US4781601046', 'US0378331005', 'US9311421039',
       'US46625H1005', 'US92826C8394', 'US01609W1027', 'US88034P1093',
       'NL0009805522', 'US16939P1066', 'US92343X1000', 'US0846707026',
       'US34959E1091', 'US78781P1057', 'IL0011691438', 'US7427181091',
       'US70450Y1038', 'US87105L1044', 'US79466L3024', 'US67066G1040',
       'US65339F1012'], dtype=object)

In [8]:
Transactions['stock_exchange'].unique()

array(['LSE', 'NSY', 'NDQ', 'ASE', 'DEG'], dtype=object)

In [9]:
closed_pos = pd.DataFrame()
current_pos = pd.DataFrame()
for produkt in (Transactions['product'].unique()):
    k = Transactions[Transactions['product'] == produkt]
    s = 0
    for i in range (len(k)):
        s += k.iloc[i][6]
    if s == 0:
        closed_pos = closed_pos.append(k,ignore_index = True)
    else:
        current_pos = current_pos.append(k,ignore_index = True)
    

In [10]:
items_closed = list(zip(closed_pos['isin'],closed_pos['stock_exchange'], closed_pos['exchCode'],closed_pos['trans_curr']))
items_current = list(zip(current_pos['isin'],current_pos['stock_exchange'], current_pos['exchCode'],current_pos['trans_curr']))

In [11]:
stocks_closed = []
for item in items_closed:
    stock = list(item)
    stocks_closed.append(stock)

In [12]:
stocks_current = []
for item in items_current:
    stock = list(item)
    stocks_current.append(stock)

In [13]:
for i in range (len(stocks_closed)):
    if stocks_closed[i][1] in ('NDQ', 'NSY','ASE'):
        stocks_closed[i][2] = 'US'
    elif stocks_closed[i][1] == 'LSE':
        stocks_closed[i][2] = 'LN'
    elif stocks_closed[i][0] == 'US8762142060' or stocks[i][1] == 'DEG' :
        stocks_closed[i][2] = 'US'
        stocks_closed[i][3] = 'USD'

In [14]:
for i in range (len(stocks_current)):
    if stocks_current[i][1] in ('NDQ', 'NSY','ASE'):
        stocks_current[i][2] = 'US'
    elif stocks_current[i][1] == 'LSE':
        stocks_current[i][2] = 'LN'
    elif stocks_current[i][0] == 'US8762142060' or stocks[i][1] == 'DEG' :
        stocks_current[i][2] = 'US'
        stocks_current[i][3] = 'USD'

In [15]:
jobs_closed = []
for i in range (len(stocks_closed)):
    job = {'idType': 'ID_ISIN', 'idValue': stocks_closed[i][0], 'exchCode': stocks_closed[i][2], 'currency': stocks_closed[i][3], "includeUnlistedEquities": True}
    jobs_closed.append(job)

In [16]:
jobs_current = []
for i in range (len(stocks_current)):
    job = {'idType': 'ID_ISIN', 'idValue': stocks_current[i][0], 'exchCode': stocks_current[i][2], 'currency': stocks_current[i][3],"includeUnlistedEquities": True}
    jobs_current.append(job)

In [17]:
openfigi_apikey = '844b2af4-4c56-455c-907d-3fc4b5226880'  
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

In [18]:
def map_jobs_closed(jobs_closed):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs_closed)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()
job_results_closed = map_jobs_closed(jobs_closed)

In [19]:
stocks_closed = [d['data'][0] for d in job_results_closed]

In [20]:
stocks_closed = pd.DataFrame.from_dict(stocks_closed)

In [ ]:
d_closed = pd.DataFrame(list(zip(closed_pos['date'], closed_pos['time'], closed_pos['product'], 
                                 closed_pos['isin'], closed_pos['stock_exchange'], closed_pos['execution_place'],
                                 closed_pos['number'], closed_pos['price'], closed_pos['trans_curr'], closed_pos['local_value'],
                                 closed_pos['curr_local_value'],
                                 closed_pos['price_eu'], closed_pos['account_curr'], closed_pos['exchange_rate'],
                                 closed_pos['commission'], closed_pos['commission_curr'], closed_pos['total_eu'],
                                 closed_pos['total_curr'], closed_pos['exchCode'],
                                 stocks_closed['name'], stocks_closed['ticker']
                                )), columns = ['date', 'time', 'product', 'isin', 'stock_exchange', 'execution_place',
                                                'number', 'price', 'trans_curr', 'local_value', 'curr_local_value',
                                                'price_eu', 'account_curr', 'exchange_rate', 'commission',
                                                'commission_curr', 'total_eu', 'total_curr', 'exchCode', 'figi_name',
                                                'ticker'])


In [ ]:
currency = [None]*len(d_closed)
pair_currency = [None]*len(d_closed)
d_closed['pair_currency'] = pair_currency
for i in range (len(d_closed)):
    if d_closed['trans_curr'][i] == 'USD':
        d_closed['pair_currency'][i] = 'EUR=X'
    elif d_closed['trans_curr'][i] == 'GBP':
        d_closed['pair_currency'][i] = 'GBPEUR=X'
    if d_closed['ticker'][i] == 'CSPX':
        d_closed['ticker'][i] = 'CSPX.L'
    if d_closed['ticker'][i] == 'VUSA':
        d_closed['ticker'][i] = 'VUSA.L'

In [25]:
def map_jobs_current(jobs_current):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs_current)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()
job_results_current = map_jobs_current(jobs_current)

In [26]:
stocks_current = [d['data'][0] for d in job_results_current]

In [27]:
stocks_current = pd.DataFrame.from_dict(stocks_current)

In [36]:
d_current = pd.DataFrame(list(zip(current_pos['date'], current_pos['time'], current_pos['product'], 
                                 current_pos['isin'], current_pos['stock_exchange'], current_pos['execution_place'],
                                 current_pos['number'], current_pos['price'], current_pos['trans_curr'], current_pos['local_value'],
                                 current_pos['curr_local_value'],
                                 current_pos['price_eu'], current_pos['account_curr'], current_pos['exchange_rate'],
                                 current_pos['commission'], current_pos['commission_curr'], current_pos['total_eu'],
                                 current_pos['total_curr'], current_pos['exchCode'],
                                 stocks_current['name'], stocks_current['ticker']
                                )), columns = ['date', 'time', 'product', 'isin', 'stock_exchange', 'execution_place',
                                                'number', 'price', 'trans_curr', 'local_value', 'curr_local_value',
                                                'price_eu', 'account_curr', 'exchange_rate', 'commission',
                                                'commission_curr', 'total_eu', 'total_curr', 'exchCode', 'figi_name',
                                                'ticker'])

In [83]:
pair_currency = [None]*len(d_current)
d_current['pair_currency'] = pair_currency

In [84]:
for i in range (len(d_current)):
    if d_current['trans_curr'][i] == 'USD':
        d_current['pair_currency'][i] = 'EUR=X'
    elif d_current['trans_curr'][i] == 'GBP':
        d_current['pair_currency'][i] = 'GBPEUR=X'
    if d_current['ticker'][i] == 'BRK/B':
        d_current['ticker'][i] = 'BRK-B'


In [76]:
def parse_stock(stock,period1,period2):
    stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
    params = {
        'period1' : period1 ,
        'period2' : period2,
        'interval' : '1d',
        'events' : 'history'
    }
    response_stock = requests.get(stock_url.format(stock), params = params,headers={'User-agent': 'Mozilla/5.0'})
    file_stock = StringIO(response_stock.text)
    reader_stock = csv.reader(file_stock)
    data_stock = pd.DataFrame (reader_stock, columns = ['Data','Open','High','Low','Close','Adj Close','Volume']).drop([0])
    data_stock['Data'] = pd.to_datetime(data_stock['Data'], yearfirst=True)
    data_stock['Adj Close'] = pd.to_numeric(data_stock['Adj Close'])
    data_stock['Close'] = pd.to_numeric(data_stock['Close'])
    return data_stock

In [77]:
def parse_currency(currency,period1,period2):
    currency_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
    params = {
        'period1' : period1 ,
        'period2' : period2,
        'interval' : '1d',
        'events' : 'history'
    }
    response_currency = requests.get(currency_url.format(currency), params = params,headers={'User-agent': 'Mozilla/5.0'})
    file_currency = StringIO(response_currency.text)
    reader_currency = csv.reader(file_currency)
    fx = pd.DataFrame (reader_currency, columns = ['Data','Open','High','Low','Close','Adj Close','Volume']).drop([0])
    fx['Data'] = pd.to_datetime(fx['Data'], yearfirst=True)
    fx['Adj Close'] = pd.to_numeric(fx['Adj Close'])
    fx['Close'] = pd.to_numeric(fx['Close'])
    return fx

In [78]:
def profit_closed(merge, stock_degiro,for_zero):
    amount_of_bought_stocks = 0
    amount_of_sold_stocks = 0
    sum_of_bought_stocks = 0
    sum_of_sold_stocks = 0
    opłaty_trans = 0
    sum_of_bought_stocks_without_currency = 0
    sum_of_sold_stocks_without_currency = 0
    n = []
    roi_brutto_closed = [0]*for_zero
    for i in range (len(merge)):
        for j in range(len(stock_degiro)):
            if merge.iloc[i][0].date() == stock_degiro.iloc[j][0].date() and stock_degiro.iloc[j,6] > 0 :
                amount_of_bought_stocks += stock_degiro.iloc[j,6]
                sum_of_bought_stocks += (-1)*stock_degiro.iloc[j,11]
                sum_of_bought_stocks_without_currency += stock_degiro.iloc[j,7]
                opłaty_trans += (-1)*stock_degiro.iloc[j,14]
                a = -stock_degiro.iloc[j,11]/stock_degiro.iloc[j,6]
                for k in range(stock_degiro.iloc[j,6]):
                    n.append(a)
                average_price = sum_of_bought_stocks/amount_of_bought_stocks 
                average_price_without_currency = sum_of_bought_stocks_without_currency / amount_of_bought_stocks
            if data_stock.iloc[i][0].date() == stock_degiro.iloc[j][0].date() and stock_degiro.iloc[j,6] < 0:
                amount_of_sold_stocks += stock_degiro.iloc[j,6]
                sum_of_sold_stocks = stock_degiro.iloc[j,11]
                sum_of_sold_stocks_without_currency = stock_degiro.iloc[j,7]
                opłaty_trans += (-1)*stock_degiro.iloc[j,14]
                for l in range((-1)*stock_degiro.iloc[j,6]):
                    n.pop(0)
                if len(n) != 0:
                    average_price = sum(n)/len(n)
                else:
                    gains = ((-1)*sum_of_sold_stocks/amount_of_sold_stocks - average_price)/average_price
                    gains_without_currency = ((merge.iloc[i,1]) - average_price_without_currency)/average_price_without_currency
                    qqq = [gains_without_currency,merge.iloc[i][0].date()]
                    roi_brutto_closed.append(gains)
                    return roi_brutto_closed


        gains = ((merge.iloc[i,1]*merge.iloc[i,2]) - average_price)/average_price
        gains_without_currency = ((merge.iloc[i,1]) - average_price_without_currency)/average_price_without_currency
        roi_brutto_closed.append(gains)
    return roi_brutto_closed

In [101]:
def profit_current(merge, stock_degiro):
    amount_of_bought_stocks = 0
    amount_of_sold_stocks = 0
    sum_of_bought_stocks = 0
    sum_of_sold_stocks = 0
    sum_of_bought_stocks_without_currency = 0
    sum_of_sold_stocks_without_currency = 0
    opłaty_trans = 0
    n = []
    for i in range (len(merge)):      
        l = len(stock_degiro[stock_degiro['date'].dt.date == merge.iloc[i][0].date()])
        if l != 0:
            for j in range(l):
                stock_deg = stock_degiro[stock_degiro['date'].dt.date == merge.iloc[i][0].date()].reset_index(drop=True)
                if  stock_deg.iloc[j,6] > 0:
                    amount_of_bought_stocks += stock_deg.iloc[j,6]
                    sum_of_bought_stocks += (-1)*stock_deg.iloc[j,11]
                    sum_of_bought_stocks_without_currency += stock_deg.iloc[j,7]
                    a = -stock_deg.iloc[j,11]/stock_deg.iloc[j,6]
                    opłaty_trans += (-1)*stock_deg.iloc[j,14] 

                for k in range(stock_deg.iloc[j,6]):
                    n.append(a)

                average_price = sum_of_bought_stocks/amount_of_bought_stocks   
                average_price_without_currency = sum_of_bought_stocks_without_currency / amount_of_bought_stocks

                if  stock_deg.iloc[j,6] < 0:
                    amount_of_sold_stocks += stock_deg.iloc[j,6]
                    sum_of_sold_stocks = stock_deg.iloc[j,11]
                    sum_of_sold_stocks_without_currency = stock_deg.iloc[j,7]
                    opłaty_trans += (-1)*stock_deg.iloc[j,14]
                    for m in range((-1)*stock_deg.iloc[j,6]):
                        n.pop(0)
                    average_price = sum(n)/len(n)

        gains = ((merge.iloc[i,1]*merge.iloc[i,2]) - average_price)/average_price
        gains_without_currency = ((merge.iloc[i,1]) - average_price_without_currency)/average_price_without_currency
        infl_of_currency = gains - gains_without_currency
        roi_brutto_current.append(gains)
    return roi_brutto_current

In [40]:
closed = d_closed.groupby('ticker')
current = d_current.groupby('ticker')

In [102]:
start = time.time()
k = 0
start_point = str(int(d_current['date'][0].timestamp()))
period2 = str(int(datetime.now().timestamp()))
fx = parse_currency(d_current['pair_currency'][k],start_point,period2)
roi_brutto_current = {}
for key in list(current.groups.keys()):
    stock_degiro = current.get_group(key)    
    period1 = str(int(stock_degiro.iloc[0][0].timestamp()))
    data_stock = parse_stock(key,period1,period2)
    if stock_degiro['pair_currency'].iloc[k] != 'EUR=X':
        fx = parse_currency(d_current['pair_currency'][k],period1,period2)
    
    merge = pd.merge(data_stock,fx,how='inner', on='Data').drop(['Open_x','High_x','Low_x','Close_x','Volume_x','Open_y','High_y','Low_y','Close_y','Volume_y'],axis=1)
    merge = merge.rename(columns = {'Data':'Date','Adj Close_x':'Stock_adj_close','Adj Close_y':'curr_adj_close'})
    roi_brutto_current[key] = profit_current(merge, stock_degiro)
    k += 1
end = time.time()

AttributeError: 'dict' object has no attribute 'append'

In [ ]:
d_cur = pd.DataFrame.from_dict(roi_brutto_current)

In [ ]:
d_cl = pd.DataFrame.from_dict(roi_brutto_closed)

In [226]:
keys_cl = list(s_closed.keys())

In [227]:
start = time.time()
k = 0
start_point = str(int(Transactions['Data'][0].replace(hour=0, minute=0, second=0, microsecond=0).timestamp()))
period2 = str(int(datetime.now().timestamp()))
fx = parse_currency(d_current['pair_currency'][k],start_point,period2)
roi_brutto_closed = {}
for key in s_closed:
    stock_degiro = s_closed[key]
    stock_degiro['Data'] = pd.to_datetime(stock_degiro['Data'],dayfirst=True)
    stock_degiro = stock_degiro.sort_values('Data')
    period1 = str(int(stock_degiro.iloc[0][0].timestamp()))
    period2 = str(int(stock_degiro.iloc[-1][0].timestamp()))
    data_stock = parse_stock(key,period1,period2)
    if key == keys_cl[0]:
        for_zer = len(data_stock)
    for_zero = for_zer - len(data_stock)
    if d_closed['pair_currency'][k] != 'EUR=X':
        fx = parse_currency(d_current['pair_currency'][k],period1,period2)
    
    merge = pd.merge(data_stock,fx,how='inner', on='Data').drop(['Open_x','High_x','Low_x','Close_x','Volume_x','Open_y','High_y','Low_y','Close_y','Volume_y'],axis=1)
    merge = merge.rename(columns = {'Data':'Date','Adj Close_x':'Stock_adj_close','Adj Close_y':'curr_adj_close'})

    roi_brutto_closed [key] = profit_closed(merge, stock_degiro,for_zero)
    k += 1
end = time.time()

ValueError: 7 columns passed, passed data had 1 columns

In [233]:
s_current['GILD']

,Data,Czas,Produkt,ISIN,Giełda referenc,Miejsce wykonania,Liczba,Kurs,waluta_pozycji,Wartość lokalna,waluta_wartości_lok,Wartość,waluta_zak,Kurs wymian,Opłata transakcyjna,Opł_trans_waluta,Razem,waluta_zak_razem,Identyfikator zlecenia,exchCode
0,2020-04-17 15:30:00,15:30,GILEAD SCIENCES INC.,US3755581036,NDQ,XNAS,1,85.36,USD,-85.36,USD,-78.50,EUR,1.0874,-0.5,EUR,-79.00,EUR,20293a07-deba-4c03-8af8-a66edd0f0079,None
1,2020-07-24 18:09:00,18:09,GILEAD SCIENCES INC.,US3755581036,NDQ,NaN,1,73.93,USD,-73.93,USD,-63.56,EUR,1.1632,-0.5,EUR,-64.06,EUR,4c6260f2-b39f-47cc-9c43-a0fe56be441b,None
2,2020-07-29 21:50:00,21:50,GILEAD SCIENCES INC.,US3755581036,NDQ,NaN,1,72.95,USD,-72.95,USD,-61.91,EUR,1.1783,-0.5,EUR,-62.41,EUR,1d830aed-d716-47bb-829b-09e46dac3c0e,None
3,2020-08-05 18:31:00,18:31,GILEAD SCIENCES INC.,US3755581036,NDQ,NaN,1,69.49,USD,-69.49,USD,-58.48,EUR,1.1882,-0.5,EUR,-58.98,EUR,f5c3e58a-d1d8-4dcb-b583-9dbbf9252dd3,None


In [207]:
s = {}
l = len(d_closed) + len(d_current)

# for i in range (len(d_current)):
#     degiro_name = d_current['degiro_name'][i]
#     s[d_current['ticker'][i]] = current_pos[current_pos['Produkt'] == degiro_name]
    
for j in range (l - len(d_current)):
    degiro_name = d_closed['degiro_name'][j]
    s[d_closed['ticker'][j]].append(closed_pos[closed_pos['Produkt'] == degiro_name])

KeyError: 'VUSA.L'

In [208]:
s.keys()

dict_keys([])

In [209]:
account = pd.read_csv('Account.csv')

In [210]:
account = account.rename({'Unnamed: 8': 'wartość', 'Unnamed: 10': 'saldo'},axis= 1)

In [211]:
account['wartość'] = account.wartość.str.replace(',','.').astype(float)

In [212]:
tax_div = account[account['Opis'] == 'Podatek Dywidendowy'].reset_index(drop=True)

In [213]:
tax_trans = account.loc[account['Opis'].str.contains('od transakcji', case = False, na = False)].reset_index(drop=True)

In [214]:
div = account[account['Opis'] == 'Dywidenda']
div = div.reset_index(drop=True)

In [215]:
stocks = div['Produkt'].unique()

In [216]:
d = {}
for i in range (len(stocks)):
    s = 0
    for j in range (len(div)):
        if stocks[i] == div['Produkt'][j]:
            s += div['wartość'][j]
    d[stocks[i]] = s
    
            

In [217]:
t_d = {}
for i in range(len(stocks)):
    t = 0
    for j in range(len(tax_div)):
        if stocks[i] == tax_div['Produkt'][j]:
            t += tax_div['wartość'][j]
    t_d[stocks[i]] = t

In [218]:
netto_div = {}
for key in (d):
    n = d[key] - (-t_d[key])
    netto_div[key] = n

In [219]:
t_t = {}
for i in range(len(stocks)):
    tt = 0
    for j in range(len(tax_trans)):
        if stocks[i] == tax_trans['Produkt'][j]:
            tt += tax_trans['wartość'][j]
    t_t[stocks[i]] = tt

In [220]:
period1 = str(int(Transactions['Data'][0].replace(hour=0, minute=0, second=0, microsecond=0).timestamp()))
period2 = str(int(datetime.now().timestamp()))

In [221]:
sp = parse_stock('^GSPC',period1,period2 ).reset_index(drop=True)
sp['Open'] = pd.to_numeric(sp['Open'])

In [222]:
sp['weight'] = None
sp['weight'][0] = sp.iloc[0][5]/sp.iloc[0][1]
for i in range(1,len(sp)):
    sp['weight'][i] = sp.iloc[i][5]/sp.iloc[i-1][5]

In [223]:
sp['portfolio'] = None
sp['portfolio_change'] = None
sp['top up'] = None

sp_portfolio = (-1)*Transactions.iloc[0,9]*sp['weight'][0]
sp['portfolio'][0] = sp_portfolio
s = (-1)*Transactions.iloc[0,9]
sp['top up'][0] = s
sp['portfolio_change'][0] = ((sp_portfolio - s)/s)*100

for i in range (1,len(sp)):
    l = len(Transactions[Transactions['Data'] == sp.iloc[i,0]])
    if l == 0:
        sp_portfolio = sp_portfolio* sp['weight'][i]
    else:
        for j in range(l):
            Transact = Transactions[Transactions['Data'] == sp.iloc[i,0]].reset_index(drop=True)
            s += (-1)*Transact.iloc[j,9]
            sp_portfolio += (-1)*Transact.iloc[j,9]  
        sp_portfolio = sp_portfolio* sp['weight'][i]
        portfolio_change = ((sp_portfolio - s)/s)*100
    sp['portfolio'][i] = sp_portfolio
    sp['top up'][i] = s
    sp['portfolio_change'][i] = ((sp_portfolio - s)/s)*100